In [63]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [64]:
import os
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
from sklearn.metrics import matthews_corrcoef
import csv

In [65]:
from fastai import *
from fastai.vision import *

In [66]:
from data_loader import ImageItemListCell

In [67]:
path = Path('/data/Datasets/WhiteBloodCancer/train/')
test_path = Path('/data/Datasets/WhiteBloodCancer/test/')
test_files = get_files(test_path)
test = ImageItemListCell(test_files)

In [69]:
for model in path.glob('*.pkl'):

    learn = load_learner(path, model, test)
    learn.model.float()
    
    # Use TTA
    preds_test_tta, y_test_tta = learn.TTA(ds_type=DatasetType.Test, beta=0.6, scale=1.3)
    y_pred = np.argmax(tta_result, axis=1)
    
    # use normal prediction
    #preds_test,y_test=learn.get_preds(ds_type=DatasetType.Test)
    #y_pred = np.argmax(torch.sigmoid(preds_test), axis=1)
    
    with open(path.joinpath('{}.csv'.format(model.stem)), mode='w') as submission:
        submission_writer = csv.writer(submission, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        for fn, y in zip(learn.data.test_dl.items, y_pred):
            submission_writer.writerow([fn.name, y]) #learn.data.classes[y]

In [99]:
ensemble = np.zeros(shape=(len(learn.data.test_dl.items)))
model_count = len(list(path.glob('*.csv')))
for submission in path.glob('*.csv'):
    
    with open(submission, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for i, row in enumerate(reader):
            ensemble[i] += int(row[1])

result = np.rint(ensemble / model_count).astype(int)
with open(path.joinpath('final.csv'), mode='w') as submission:
    submission_writer = csv.writer(submission, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for fn, y in zip(learn.data.test_dl.items, result):
        submission_writer.writerow([fn.name, learn.data.classes[y]])